# Numpy (Часть 3)


> 🚀 В этой практике нам понадобятся: `numpy==1.26.4` 

> 🚀 Установить вы их можете с помощью команды: `%pip install numpy==1.26.4` 


## Содержание

* [Broadcasting (трансляция)](#Broadcasting-трансляция)
* [Изменение размеров массива (Reshape)](#Изменение-размеров-массива-Reshape)
* [Объединение массивов](#Объединение-массивов)
  * [Задание - объединяем](#Задание---объединяем)
  * [Задание - расширяем](#Задание---расширяем)
* [Сортировка массива](#Сортировка-массива)
  * [Задание - покопаемся в доках](#Задание---покопаемся-в-доках)
  * [Задание - никуда без сортировки](#Задание---никуда-без-сортировки)
* [Задачки](#Задачки)
* [Полезные ссылки](#Полезные-ссылки)


В этом ноутбуке:
- Broadcasting - операции между массивами неодинаковых размеров
- Изменение формы массива  - reshape
- Объединение массивов - concatenate
- Сортировка по элементам и индексам - sort, argsort

Теперь рассмотрим некоторые "рутинные" процедуры работы с массивами, которые пригодятся в дальнейшем. За более полной документацией сюда https://numpy.org/doc/stable/reference/routines.array-manipulation.html

In [ ]:
import numpy as np

## Broadcasting (трансляция)

Broadcasting - это механизм  выполнения математических операций с массивами *неодинаковых* размеров. Тема достаточно "широкая" =)

На деле механизм очень полезен и его понять его стоит, так как он является одним из мощных инструментов библиотеки numpy. Допустим, мы имеет вектор $(4,)$, матрицу $(3, 4)$ и хотим умножить этот вектор на каждую строку матрицы. Самый простой способ - пройти в цикле по строкам:

In [ ]:
vec = np.arange(4)
mtrx = np.ones((3, 4))
print(vec)
print(mtrx)

In [ ]:
result = np.empty_like(mtrx)

for i_row in range(mtrx.shape[0]):
    result[i_row, :] = mtrx[i_row, :] * vec

print(result)

С другой стороны, можно склонировать вектор по размеру матрицы, чтобы затем просто перемножить поэлементно две матрицы.

In [ ]:
# Функция np.tile() повторяет массив то количество раз и по тем осям,
#   которые заданы в аргументе reps
vec_m = np.tile(vec, reps=(3, 1))
print(vec_m)
print(vec_m * mtrx)

А можно вспомнить про броадкастинг и просто умножить вектор на матрицу:

In [ ]:
# Умножение, используя броадкастинг
print(vec * mtrx)

Таким образом, броадкастинг - это механизм распространения элементов операций по разметностям, по которым количество элементов не соотносится с другим операндом. В последней ячейке вектор был продублирован на каждую строку так, что размерности соотносятся.

> Обязательно прочитайте примеры броадкастинга в [статье](https://numpy.org/doc/stable/user/basics.broadcasting.html).

Броадкастинг также работает и при умножении скаляра на массив:

In [ ]:
print(2 * mtrx)

Чтобы больше понять правила броадкастинга, посмотрим на результаты операций 3D матриц:

In [ ]:
x = np.ones((4, 1, 1))
y = np.random.randint(0, 5, size=(4, 3, 2))

print((x * y).shape)

In [ ]:
x = np.ones((3, 1))
y = np.random.randint(0, 5, size=(4, 3, 2))

print((x * y).shape)

In [ ]:
x = np.ones((1, 3, 2))
y = np.random.randint(0, 5, size=(5, 1, 2))

print((x * y).shape)

В последних примерах можно увидеть, что размерности должны быть соотносимы (количества размерностей равны). В случае неравенства количества размерностей новые размерности со значением 1 добавляются слева ($(3, 1) \rightarrow (1, 3, 1)$ в одном из примеров).

> Добавление размерности было и в случае умножения вектора на матрицу: $(4,)*(3, 4) \rightarrow (1, 4)*(3, 4)$

Броадкастинг делается по тем осям, которые у одного операнда имеют размерность 1 путем копирования.

> В нашем случае $(1, 4)*(3, 4) \rightarrow (3, 4)*(3, 4)$

Осознать броадкастинг сразу не совсем просто, но главное не забывать, что такой инструмент есть и им нужно пользоваться во избежание лишнего кода.

## Изменение размеров массива (Reshape)

Ещё одна интересная тема, которую стоит рассмотреть. Мы уже касались вопроса приведения массива к выпрямленному виду - это один из видов изменения размерности массива.

По сути, размерность массива - это порядок выстроенных элементы в нём. Изменение размерности - это изменение структуры **без изменения элементов или порядка**.

Рассмотрим на примере:

In [ ]:
arr = np.arange(20)
print(arr)

Вот мы имеем вектор из 20 элементов в порядке возрастания. Теперь, допустим, нам нужно сделать массив размером $(4, 5)$, в котором каждый ряд - это продолжение предыдущего по возрастанию чисел. Для начала сделаем руками: 

In [ ]:
mtrx = np.ndarray((4, 5), dtype=np.int32)
mtrx[0, :] = arr[:5]
mtrx[1, :] = arr[5:10]
mtrx[2, :] = arr[10:15]
mtrx[3, :] = arr[15:]

print(mtrx)

Получилось как надо, но есть два недостатка:
- Если элементов будет больше и нужен будет другой размер - код не универсален;
- Приходится расставлять данные вручную.

Теперь взглянем, как работает метод `ndarray.reshape()`:

In [ ]:
print(arr.reshape((4, 5)))

В результате происходит тоже самое. То есть, элементы расположились в порядке размещения по размерностям: начиная с первой строки (нулевая размерность) заполняется по колонкам (первая размерность), а как только заполнение по колонкам закончено - переходим на следующий ряд.

Таким образом, в какую бы размерность не происходило изменение размера - расположение будет придерживаться этого правила.

Для лучшего понимания посмотрим на другие представления:

In [ ]:
print(arr.reshape((2, 10)))
print(arr.reshape((5, 4)))
print(arr.reshape((20, 1)))

Таким образом, можно приводить к любой размерности с учетом правила:
> Количество элементов должно сохраняться

Но бывают ситуации, когда мы точно не знаем количество элементов, но нам нужно точно создать, например, 4 строки в матрице. В этом поможет неопределенная размерность, которая задается числом -1:

In [ ]:
print(arr.reshape((4, -1)))

Таким образом можно задать, что матрица должна быть 2D с фиксированным количеством строк, но без фиксации на количество столбцов. Конечно же, если в исходном будет 21 элемент, то такой трюк не сработает, потому что 21 не делится на 4 без остатка.

## Объединение массивов

При работе с массивами бывают случаи, когда несколько массивов необходимо объединить в один. Для этого поможет функция `np.concatenate()` и понимание ее основных правил.

In [ ]:
x = np.arange(10).reshape((2, 5))
y = np.arange(15).reshape((3, 5))

print(x)
print(y)

Для объединения массивов необходимо, чтобы одна из размерностей имела одинаковое количество элементов. Это достаточно логичное требование задаётся аргументом `axis` в функции. Для запоминания направления линии соединения и описыавется числом - "0"-строка, "1" - столбец.

In [ ]:
print(np.concatenate((x, y), axis=0))

Обратите внимание, что массивы передаются в виде кортежа, что позволяет объединять два и более массивов.

### Задание - объединяем

Объедините три массива вдоль колонок:

In [ ]:
x = np.arange(9).reshape((3, 3))
y = np.arange(12).reshape((3, 4))
z = np.arange(15).reshape((3, 5))

print(x)
print(y)
print(z)

# TODO - получите объединенный массив:
# [[ 0  1  2  0  1  2  3  0  1  2  3  4]
#  [ 3  4  5  4  5  6  7  5  6  7  8  9]
#  [ 6  7  8  8  9 10 11 10 11 12 13 14]]

### Задание - расширяем

Напишите реализацию добавления стобца единиц к матрице слева:

In [ ]:
x = np.arange(1, 10).reshape((3, 3))
print(x)

# TODO - получить матрицу со столбцом единиц слева
# [[1 1 2 3]
#  [1 4 5 6]
#  [1 7 8 9]]

## Сортировка массива

Numpy как серьёзная библиотека работы с массивами имеет также и функционал сортировки. Для этого есть функция `np.sort()`, которая производит сортировку элементов и возвращает отсортированный массив:

In [ ]:
unsorted_arr = np.array([2, 4, 5, 1, 2, 7, 5, 0, -1, 3, 5])
sorted_arr = np.sort(unsorted_arr)

print(unsorted_arr)
print(sorted_arr)

Помимо явной сортировки существует также функция `np.argsort()`, которая возвращает не отсортированный массив, а массив индексов исходного массива в порядке, который даёт отсортированный исходный. Посмотрим:

In [ ]:
sorted_indices = np.argsort(unsorted_arr)
print(sorted_indices)

# А теперь воспользуемся индексами и отобразим исходный массив с индексацией
print(unsorted_arr[sorted_indices])

### Задание - покопаемся в доках

По такому же принципу именования найдите функцию в документации, которая возвращает индекс максимального/минимального (в завимости от варианта) элемента в массиве.

Воспользуйтесь найденной функцией и найдите индексы максимальных элементов в каждом столбце 2D матрицы: 

In [ ]:
arr = np.array([
    [11, -1, 1, 3, 5],
    [6, 1, 0, -1, 10],
    [1, 3, 2, 3, 6],
    [5, 2, 7, 8, 9],
])

print(arr)

# TODO - напишите код поиска индексов максимальных/минимальных 
#           элементов в массиве по столбцам: [0 2 3 3 1]/[2 0 1 1 0]

### Задание - никуда без сортировки

Напишите код сортировки массива по столбцам:

In [ ]:
arr = np.array([
    [11, 2, 1, 3, 5],
    [6, 2, 0, -1, 10],
    [1, 3, 2, 3, 5],
    [5, 2, 7, 8, 9],
])
print(arr)

# TODO - напишите код сортировки:
# [[ 1  2  0 -1  5]
#  [ 5  2  1  3  5]
#  [ 6  2  2  3  9]
#  [11  3  7  8 10]]

## Задачки

Создайте массив, состоящий из случайных элементов в диапазоне $[-10; 20]$ размером $(5, 7)$:

In [ ]:
# TODO - создайте массив со случайными целочисленными элементами

Добавьте границу в виде нулей со всех сторон 2D массива:

In [ ]:
data = np.arange(1, 16).reshape((3, 5))
print(data)

# TODO - реализуйте добавление нулей со всех сторон массива:
# [[ 0  0  0  0  0  0  0]
#  [ 0  1  2  3  4  5  0]
#  [ 0  6  7  8  9 10  0]
#  [ 0 11 12 13 14 15  0]
#  [ 0  0  0  0  0  0  0]]

Разберитесь с функцией `np.clip()` и произведите ограничение массива, чтобы в нем значения были не более шести и не менее двух:

In [ ]:
data = np.arange(1, 16).reshape((3, 5))
print(data)

# TODO - ограничьте значения массива
# [[2 2 3 4 5]
#  [6 6 6 6 6]
#  [6 6 6 6 6]]

Произведите модификацию массива так, чтобы значение в колонке результата соответствовало исходному значению, возведенному в степень индекса колонки:

In [ ]:
data = np.full((3, 5), fill_value=2, dtype=np.int32)
print(data)

# TODO - возведите значения в колонках в степень индекса колонки
# [[ 1  2  4  8 16]
#  [ 1  2  4  8 16]
#  [ 1  2  4  8 16]]

Умножьте каждую колонку на её индекс:

In [ ]:
data = np.full((3, 5), fill_value=2, dtype=np.int32)
print(data)

# TODO - Умножьте каждый столбец на свой индекс
# [[0 2 4 6 8]
#  [0 2 4 6 8]
#  [0 2 4 6 8]]

Умножьте каждый чётный ряд в матрице на вектор:

In [ ]:
data = np.repeat(np.arange(1, 8), 4).reshape(7, 4)
vec = np.array([1, 2, 1, 2])
print(data)

# TODO - умножьте каждый четный ряд в матрице на вектор
# [[ 1  2  1  2]
#  [ 2  2  2  2]
#  [ 3  6  3  6]
#  [ 4  4  4  4]
#  [ 5 10  5 10]
#  [ 6  6  6  6]
#  [ 7 14  7 14]]

Определите индексы двух наибольших элементов в матрице по каждому ряду:

In [ ]:
data = np.array([[20, 22,  2, 14, 25,  8],
                [ 7, 25, 23,  3, 22, 15],
                [ 8, 16,  9, 22,  0,  9],
                [ 4, 24, 24, 28,  3, 40]])
print(data)

# TODO - определите индексы двух наибольших элементов по каждому ряду
# [[4 1]
#  [1 2]
#  [3 1]
#  [5 3]]

Выберите две случайные колонки из массива:

<details><summary>Подсказка 1</summary>
Для случайного выбора полезно воспользоваться функцией `np.random.choice()`
</details>

<details>
<summary>Подсказка 2</summary>
`np.random.choice()` принимает на вход 1D массив, так что для выбора можно создать массив индексов колонок (`range()` с количеством колонок) и из него функция выберет два случайных индекса
</details>

In [ ]:
data = np.array([[20, 22,  2, 14, 25,  8],
                [ 7, 25, 23,  3, 22, 15],
                [ 8, 16,  9, 22,  0,  9],
                [ 4, 24, 24, 28,  3, 40]])
print(data)

# TODO - выберите случайные две колонки из массива
# *Например,
# [[22  8]
#  [25 15]
#  [16  9]
#  [24 40]]

Создайте массив размером $(10, 3)$, который состоит из повторяющихся рядов исходного массива:

In [ ]:
data = np.arange(9).reshape(3, 3)
print(data)

# TODO - создайте массив из повторяющихся рядов размером (10, 3)
# *Например,
# [[3 4 5]
#  [6 7 8]
#  [0 1 2]
#  [6 7 8]
#  [3 4 5]
#  [6 7 8]
#  [6 7 8]
#  [6 7 8]
#  [6 7 8]
#  [0 1 2]]

Соберите массив, состоящий из элементов 2-й и 4-й колонок и строк с 1-й по 3-ю (**цифры указаны с учётом индексации с 0-ля!**):

In [ ]:
data = np.array([[20, 22,  2, 14, 25,  8],
                [ 7, 25, 23,  3, 22, 15],
                [ 8, 16,  9, 22,  0,  9],
                [ 14, 9,  33, 21,  7,  6],
                [ 4, 24, 24, 28,  3, 40]])
print(data)
# TODO - соберите матрицу из конкретных элементов массива
# [[23 22]
#  [ 9  0]
#  [33  7]]

Инвертируйте порядок элементов в последних двух колонках:

In [ ]:
data = np.array([[20, 22,  2, 14, 25,  8],
                [ 7, 25, 23,  3, 22, 15],
                [ 8, 16,  9, 22,  0,  9],
                [ 14, 9,  33, 21,  7,  6],
                [ 4, 24, 24, 28,  3, 40]])
print(data)
# TODO - инвертируйте порядок последних двух колонок
# [[20 22  2 14  3 40]
#  [ 7 25 23  3  7  6]
#  [ 8 16  9 22  0  9]
#  [14  9 33 21 22 15]
#  [ 4 24 24 28 25  8]]

## Полезные ссылки
* [More than 15 jupyter notebooks to learn Numpy](https://www.kaggle.com/getting-started/115421)
